# Nested JSON to DataFrame

This example notebook shows you how to flatten nested JSON, using only `$"column.*"` and `explode` methods.

Start by passing the sample JSON string to the reader.

In [ ]:
val json ="""
{
    "id": "0001",
    "type": "cinnamon roll",
    "name": "Pastry",
    "ppu": 0.70,
    "batters":
        {
            "batter":
                [
                    { "id": "1001", "type": "Normal" },
                    { "id": "1002", "type": "Chocolatey" },
                    { "id": "1003", "type": "Blueberry" },
                    { "id": "1004", "type": "Raisins" }
                ]
        },
    "topping":
        [
            { "id": "5001", "type": "None" },
            { "id": "5002", "type": "Sugar-Free Cinammon" },
            { "id": "5005", "type": "Sugar" },
            { "id": "5007", "type": "Powdered Sugar" },
            { "id": "5006", "type": "Chocolate with Sprinkles" },
            { "id": "5003", "type": "Cinammon Sugar" },
            { "id": "5004", "type": "Maple" }
        ]
}
"""

Add the JSON string as a collection type and pass it as an input to `spark.createDataset`. This converts it to a DataFrame. The JSON reader infers the schema automatically from the JSON string.

This sample code uses a list collection type, which is represented as `json :: Nil`. You can also use other Scala collection types, such as Seq (Scala Sequence).

In [4]:
import org.apache.spark.sql.functions._
import spark.implicits._
val DF= spark.read.json(spark.createDataset(json :: Nil))

Display the DataFrame to view the current state.



In [5]:
display(DF)

Use `$"column.*"` and `explode` methods to flatten the struct and array types before displaying the flattened DataFrame.

In [6]:
display(DF.select($"id" as "main_id",$"name",$"batters",$"ppu",explode($"topping")) // Exploding the topping column using explode as it is an array type
        .withColumn("topping_id",$"col.id") // Extracting topping_id from col using DOT form
        .withColumn("topping_type",$"col.type") // Extracting topping_tytpe from col using DOT form
        .drop($"col")
        .select($"*",$"batters.*") // Flattened the struct type batters tto array type which is batter
        .drop($"batters")
        .select($"*",explode($"batter"))
        .drop($"batter")
        .withColumn("batter_id",$"col.id") // Extracting batter_id from col using DOT form
        .withColumn("battter_type",$"col.type") // Extracting battter_type from col using DOT form
        .drop($"col")
       )